## Домашняя работа №2.

Содержание

In [ ]:
# 1 Загрузка путей и данных о видео
import pathlib

DATA_PATH = pathlib.Path().resolve()
print(DATA_PATH)

In [ ]:
# 2 импорт необходимых библиотек

import googleapiclient.discovery as api
import pandas as pd
import numpy as np

In [ ]:
# 3 загрузка данных

df_korean = pd.read_excel('Video_characteristics_КорейскаяКосметика.xlsx')

In [ ]:
# 4 просмотр данных

df_korean.head()

In [ ]:
# 6 импорт описательных статистик
from randan.descriptive_statistics import ScaleStatistics, OrdinalStatistics, NominalStatistics

In [ ]:
# 7 Выделение необходимых переменных

var_list = ['statistics.viewCount',
           'statistics.likeCount',
            'statistics.dislikesCount',
           'statistics.commentCount']

In [ ]:
# 8 копирование в отдельный DF для манипуляций и срезов данных

data = df_korean.copy()


In [ ]:
# 9 сокращение датасета

data = data[data['statistics.viewCount'] > 0]

### Задача 1. Комментарии аудитории и длительность видео

#### Исследовательский вопрос

Видео какой длительности вызывают наибольшую вовлеченность по комментариям?

#### Содержательная гипотеза:

**Видео с бОльшей длительностью будут вызывать более сильную реакцию по разрезу комментариев, поскольку люди потратили время на изучение материала и могут его откомментировать**

#### Статистические гипотезы:


**H0** Связь между длительностью видео и количеством комментариев отсутствует
**H1** Между длительностью и количеством комментариев есть связь и она положительная

In [ ]:
# 10 Оценка диапазона длительности видео
import plotly.express as px

fig = px.histogram(data, x='durationSec', nbins=500)

fig.show()

In [ ]:
# 11 Оценка связи между категориями с помощью диаграммы рассеивания

vars = ['durationSec', 'statistics.commentCount']


fig = px.scatter(data, x=vars[0], y=vars[1], trendline='ols')

fig.show()

In [ ]:
# 12 Подсчет статистик

from randan.descriptive_statistics import ScaleStatistics

ss = ScaleStatistics(data, vars)
ss.summary()

In [ ]:
# 13 Проверка на связь между признаками

from randan.bivariate_association import Correlation

corr = Correlation(data, vars, show_results=False)
corr.show_results()

### Интерпретация


Корреляция между признаками низкая (0.235) - связи нет.

Содержательно - длительные обзоры не влияют на количество комментариев к ним.

### Задача 2. Оценка влияния категории видео на вовлеченность

#### Исследовательский вопрос

Какая категория видео генерирует наибольшую вовлеченность?

#### Содержательная гипотеза:

**Категории блогов и лайфстайл видео направлены на создание пользовательской реакции, следовательно, они должны вызывать большую вовлеченность аудитории**

#### Статистические гипотезы:

**H0: Вовлеченность в разных категориях видео не отличается**
**H1: Вовлеченность в разных категориях видео отличатся на статистически значимом уровне**

# Построение метрики вовлеченности

### Формула для вовлеченности

$$
{involvement} = \frac{likesCount + dislikesCount + commentCount}{viewCount}
$$


In [ ]:
# 14 Проверка на связь между признаками

from randan.bivariate_association import Correlation

corr = Correlation(data, var_list)

corr.sort_correlations()

**Корреляция везде не отрицательная и статистически значимая (p-value < 0.05)- переменные можно использовать в расчетах вовлеченности**

In [ ]:
# 15 Визуализация взаимосвязи пар переменных с помощью диаграммы рассеяния
import plotly.express as px
from itertools import combinations

comb_list = combinations(var_list, r=2)

for pair in comb_list:

    fig = px.scatter(data, x=pair[0], y=pair[1], trendline="ols")
    fig.show()


### Расчет вовлеченности

In [ ]:
# 16 Переменная "вовлечённость"

data['involvement.nominator'] = data['statistics.likeCount']\
                                    + data['statistics.commentCount'] + data['statistics.dislikesCount']

data['involvement'] = data['involvement.nominator'] / data['statistics.viewCount']


data[['involvement'] + var_list]

### Статистическая оценка

In [ ]:
# 17 Статистика по вовлеченности

ss = ScaleStatistics(data[['involvement']],
                     normality_test=True)

In [ ]:
# 18 График распределения Involvement

import plotly.express as px

fig = px.histogram(data, y="involvement",
                   marginal="box")
fig.show()

In [ ]:
# 19 Типичные видео по вовлеченности - 1%

var = 'involvement'

involvement_lower = np.nanpercentile(data[var], 49.5)
involvement_upper = np.nanpercentile(data[var], 50.5)



data[(data[var] >= involvement_lower) & (data[var] <= involvement_upper)]

In [ ]:
# 20 Первичный анализ номинальной переменной - categoryName

from randan.descriptive_statistics import NominalStatistics

var = 'categoryName'

ns = NominalStatistics(data, [var])

freq_table = ns.frequencies()[var]

In [ ]:
# 21 Частотный график появления категорий

px.bar(freq_table, freq_table['N'])

In [ ]:
# 22 Для анализа возьмем ограниченный список категорий с выборкой от 10 видео (не включительно)

categories = ['People & Blogs', 'Howto & Style', 'Entertainment', 'Film & Animation']

data_estim = data.query('categoryName in @categories')

In [ ]:
# 23

inv_borders = [
                       np.nanpercentile(data['involvement'], 25),
                       np.nanpercentile(data['involvement'], 50),
                       np.nanpercentile(data['involvement'], 75),
                       np.nanpercentile(data['involvement'], 100)]


def make_group_from_borders(involvement):
    if involvement < inv_borders[0]:
        return '1 Low involvement'
    elif inv_borders[0] <= involvement < inv_borders[1]:
        return '2 Lower medium involvement'
    elif inv_borders[1] <= involvement < inv_borders[2]:
        return '3 Higher medium involvement'
    else:
        return '4 High involvement'

data_estim['involvement.group'] = data_estim['involvement'].apply(make_group_from_borders)

In [ ]:
# 24

var = 'involvement.group'

ns = NominalStatistics(data_estim, [var])

freq_table = ns.frequencies()[var]

In [ ]:
# 25

from randan.bivariate_association import Crosstab

ctab = Crosstab(data_estim, 'categoryName', 'involvement.group')

In [ ]:
# 26 P-value

print('P-value', ctab.pvalue)
print('Level of Significance', ctab.sig_level)

ctab.pvalue < ctab.sig_level

#### Интерпретация

Основываясь на данных теста и наблюдаемых частотах, мы имеем основания отклонить нулевую гипотезу (p-value < level of significance)

Таким образом, мы принимаем ***альтернативную гипотезу о разности уровня вовлеченности в разных группах***

Соответственно, мы можем судить об общем уровне вовлеченности в нашей теме на основе различий средних значений нашей выборки

In [ ]:
# 27 Сводная таблица для значений просмотров в категориях

view_korean = data_estim.pivot_table(index='categoryName', values='involvement', aggfunc=[np.mean])

display(view_korean.sort_values(by=[('mean', 'involvement')] ,ascending=False))